<a href="https://colab.research.google.com/github/Venkat11Thadi/Machine_learning_examples/blob/main/review_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMDB Classification

In [40]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [53]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [54]:
df = pd.read_csv('all_combined.csv')

In [55]:
df.head()

,reviewId,userName,content,score,app
0,9b96fe32-d8bc-448f-bf37-eaa79506a6b6,Tugsan Topcuoglu,Waste of time,1,Mobicip
1,d5cb1046-ee6d-4fc5-ac03-c873e6ded9df,Roxanne LaRusso,I am extremely impressed with Mobicip's Custom...,5,Mobicip
2,fe265e2e-117d-42b2-b8ca-63fc24e551b3,Williamcadder Mutevedzi,Friendly,5,Mobicip
3,f44146dc-a7a3-445e-8183-6b153f482555,Justyna D,Does not work,1,Mobicip
4,69bc7a7b-1cca-409f-96b1-8b19288ab427,Liam Whitwam,Invades privacy,1,Mobicip


In [56]:
x = df['content'].tolist()
df['feedback'] = df['score'].apply(lambda x: 1 if x >= 3 else 0)
y = df['feedback'].tolist()

In [52]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
tokenizer = Tokenizer(num_words=3000)
tokenizer.fit_on_texts(x)
x = tokenizer.texts_to_sequences(x)

In [47]:
# Pad sequences to ensure uniform length
max_len = max(len(s) for s in x)
x = pad_sequences(x, maxlen=max_len)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [49]:
# Define the model architecture
model = Sequential()
model.add(Embedding(3000, 128, input_length=max_len))
model.add(SpatialDropout1D(0.1))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.1))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

Epoch 1/10
15/15 - 17s - loss: 0.6719 - accuracy: 0.5539 - val_loss: 0.6358 - val_accuracy: 0.6820 - 17s/epoch - 1s/step
Epoch 2/10
15/15 - 14s - loss: 0.6144 - accuracy: 0.6691 - val_loss: 0.5675 - val_accuracy: 0.6862 - 14s/epoch - 931ms/step
Epoch 3/10
15/15 - 13s - loss: 0.5359 - accuracy: 0.7319 - val_loss: 0.5092 - val_accuracy: 0.7531 - 13s/epoch - 835ms/step
Epoch 4/10
15/15 - 13s - loss: 0.4037 - accuracy: 0.8398 - val_loss: 0.4621 - val_accuracy: 0.8117 - 13s/epoch - 835ms/step
Epoch 5/10
15/15 - 13s - loss: 0.2777 - accuracy: 0.8911 - val_loss: 0.4591 - val_accuracy: 0.7824 - 13s/epoch - 887ms/step
Epoch 6/10
15/15 - 13s - loss: 0.2326 - accuracy: 0.9131 - val_loss: 0.4755 - val_accuracy: 0.8326 - 13s/epoch - 848ms/step
Epoch 7/10
15/15 - 12s - loss: 0.1690 - accuracy: 0.9455 - val_loss: 0.4783 - val_accuracy: 0.8075 - 12s/epoch - 828ms/step
Epoch 8/10
15/15 - 13s - loss: 0.1212 - accuracy: 0.9581 - val_loss: 0.5248 - val_accuracy: 0.8117 - 13s/epoch - 836ms/step
Epoch 9/10


In [51]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.8326359987258911
